In [1]:
import os
import geopandas as gpd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")
local_crs = 32636
# path to data
example_data_path = "./data"

## Input data fetching
The input "blocks" data also should be obtained from the previous example result

In [2]:
from masterplan_tools.preprocessing import DataGetter, AggregateParameters

blocks = gpd.read_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet")).to_crs(local_crs)
getter = DataGetter(blocks=blocks)

/home/vasilstar/masterplanning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## City blocks information aggregating
Here we aggregate information about buildings, greenings and parkings into the city blocks

In [4]:
buildings = gpd.read_parquet(os.path.join(example_data_path, "buildings.parquet"))
greenings = gpd.read_parquet(os.path.join(example_data_path, "greenings.parquet")).rename_geometry('geometry')
parkings = gpd.read_parquet(os.path.join(example_data_path, "parkings.parquet")).rename_geometry('geometry')

aggr_params = AggregateParameters(
  buildings=buildings,
  greenings=greenings,
  parkings=parkings
)

aggregated_blocks = getter.aggregate_blocks_info(params=aggr_params)
aggregated_blocks.head()

Restoring living area squash: 100%|██████████| 3748/3748 [00:00<00:00, 173124.80it/s]


,geometry,landuse,block_id,is_living,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
0,"POLYGON ((344212.162 6646871.964, 344244.125 6...",buildings,0,False,0.0,NaN,0.000,0.0,0.0,0.0,0.0000,6.512578e+04
1,"POLYGON ((343977.009 6647706.702, 343981.163 6...",selected_area,1,False,0.0,NaN,0.000,579.0,24381.0,0.0,0.0000,2.012788e+04
2,"POLYGON ((343986.140 6647684.963, 343986.136 6...",no_dev_area,2,False,0.0,NaN,0.000,0.0,0.0,0.0,0.0000,8.037665e+02
3,"POLYGON ((343949.114 6648643.612, 343949.109 6...",no_dev_area,3,False,0.0,0.0,0.000,56088.0,117012.0,2640.0,22608.1466,7.612824e+05
4,"POLYGON ((342699.812 6648446.184, 342699.886 6...",buildings,4,True,17747.0,16.5,64035.091,0.0,0.0,2244.0,10828.9367,1.859565e+06


## Accessibility matrix 
Accessibility matrix is generated with out intermodal ```GraphGenerator``` 

In [10]:
from masterplan_tools import GraphGenerator

city_geometry = gpd.read_parquet(os.path.join(example_data_path, 'city_geometry.parquet')).to_crs(local_crs)
transport_graph = GraphGenerator(city_geometry=city_geometry, local_crs=local_crs).get_graph()
accessibility_matrix = getter.get_accessibility_matrix(transport_graph)

/home/vasilstar/masterplanning/masterplan_tools/preprocessing/accs_matrix_calculator.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.blocks.rename(columns={"index": "id"}, inplace=True)


In [11]:
accessibility_matrix.head()

id,0,1,2,3,4,5,6,7,8,9,...,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,16.0,15.0,12.0,25.0,12.0,15.0,13.0,19.0,22.0,...,83.0,86.0,82.0,84.0,81.0,80.0,79.0,78.0,83.0,84.0
1,16.0,0.0,1.0,6.0,16.0,7.0,15.0,11.0,15.0,19.0,...,87.0,90.0,86.0,88.0,85.0,84.0,83.0,82.0,87.0,88.0
2,15.0,1.0,0.0,5.0,16.0,6.0,14.0,10.0,14.0,18.0,...,86.0,89.0,85.0,87.0,84.0,83.0,82.0,81.0,86.0,87.0
3,12.0,6.0,5.0,0.0,15.0,1.0,10.0,6.0,10.0,14.0,...,82.0,85.0,81.0,83.0,80.0,79.0,78.0,77.0,82.0,83.0
4,25.0,16.0,16.0,15.0,0.0,16.0,23.0,19.0,19.0,27.0,...,82.0,85.0,81.0,83.0,80.0,79.0,78.0,77.0,82.0,83.0


Save result for the next example

In [12]:
aggregated_blocks.to_parquet(os.path.join(example_data_path, "data_getter_blocks.parquet"))
accessibility_matrix.to_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))